In [1]:
import os
from IPython.display import Image


os.chdir("/mnt/w/game_cibog/")



In [2]:
! pwd

/mnt/w/game_cibog


In [3]:
! which hisat2

/home/mjoppich/bioGUI/progs//hisat2-2.1.0/hisat2


# Aligning the reads

In order to align the reads we use hsiat2 in parallel mode (4 threads, -p 4). We provide the reference genome (-x ./references/grch38/genome) and the unpaired reads file (-U ./reads/bulk/SUM159_24h_R1.fastq.gz).

hisat2 output is an alignment in the SAM format. In order to save space, we will construct the compressed binary format from it: BAM format. This can be achieved by piping the output from samtools into samtools with parameters view -@ 4 -bS - (-@ 4 = threads, -bS = binary output).

In order to speed up the quantification step we will sort the reads by genome position, which is done by samtools sort (again with 4 threads).

In [9]:
! hisat2 -p 4 -x ./references/grch38/genome -U ./reads/bulk/SUM159_24h_R1.fastq.gz | samtools view -@ 4 -bS - | samtools sort -@ 4 -o alignments/bulk/SUM159_24h_R1.bam -

17872256 reads; of these:
  17872256 (100.00%) were unpaired; of these:
    1101887 (6.17%) aligned 0 times
    14598579 (81.68%) aligned exactly 1 time
    2171790 (12.15%) aligned >1 times
93.83% overall alignment rate
[bam_sort_core] merging from 4 files and 4 in-memory blocks...


In [6]:
! hisat2 -p 4 -x ./references/grch38/genome -U ./reads/bulk/SUM159_24h_R2.fastq.gz | samtools view -@ 4 -bS - | samtools sort -@ 4 -o alignments/bulk/SUM159_24h_R2.bam -

40740514 reads; of these:
  40740514 (100.00%) were unpaired; of these:
    1926278 (4.73%) aligned 0 times
    33800194 (82.96%) aligned exactly 1 time
    5014042 (12.31%) aligned >1 times
95.27% overall alignment rate
[bam_sort_core] merging from 12 files and 4 in-memory blocks...


In [7]:
! hisat2 -p 4 -x ./references/grch38/genome -U ./reads/bulk/SUM159R_24h_R1.fastq.gz | samtools view -@ 4 -bS - | samtools sort -@ 4 -o alignments/bulk/SUM159R_24h_R1.bam -

43554379 reads; of these:
  43554379 (100.00%) were unpaired; of these:
    1944297 (4.46%) aligned 0 times
    36312679 (83.37%) aligned exactly 1 time
    5297403 (12.16%) aligned >1 times
95.54% overall alignment rate
[bam_sort_core] merging from 12 files and 4 in-memory blocks...


In [8]:
! hisat2 -p 4 -x ./references/grch38/genome -U ./reads/bulk/SUM159R_24h_R2.fastq.gz | samtools view -@ 4 -bS - | samtools sort -@ 4 -o alignments/bulk/SUM159R_24h_R2.bam -

28277603 reads; of these:
  28277603 (100.00%) were unpaired; of these:
    1520084 (5.38%) aligned 0 times
    23332433 (82.51%) aligned exactly 1 time
    3425086 (12.11%) aligned >1 times
94.62% overall alignment rate
[bam_sort_core] merging from 8 files and 4 in-memory blocks...


# BAM output

The BAM output shows where a read is aligned to (column 3 (chromosome) and 4 (bp-position)).

In [16]:
! cd ./alignments/bulk/ && samtools view -h SUM159R_24h_R1.bam | head -n 250 | tail -n 10

samtools view: writing to standard output failed: Broken pipe
SRR9048103.31968550	272	1	14298	1	50M	*	0	0	TCTGCCCCCACTGCCTAGGGACCAACAGGGGCAGGAGGCANTCACTGACC	HIIIIHCEGHFFF@B9IGIHFGGEFEIIIIIIIIGGGCA2#HFDFDF@@@	AS:i:-2	ZS:i:-2	XN:i:0	XM:i:1	XO:i:0	XG:i:0	NM:i:1	MD:Z:40G9	YT:Z:UU	NH:i:5
SRR9048103.14337000	256	1	14363	1	50M	*	0	0	CCTGCACAGCTAGAGATCCTTTATTAAAAGCACACTGTTGGTTTCTGCTC	@@@?DDDDHDADFGGE:EHIIIIIIGEH>?FFGIGBHIIDIDEHFCGHIE	AS:i:0	ZS:i:0	XN:i:0	XM:i:0	XO:i:0	XG:i:0	NM:i:0	MD:Z:50	YT:Z:UU	NH:i:4
SRR9048103.20836045	256	1	14369	1	1S49M	*	0	0	GCAGCTAGAGATCCTTTATTAAAAGCACACTGTTGGTTTCTGCTCAGTTC	;?;DADDB:+2<FGIFIBHHFICEFGEFGGEHGIC<:CGFDD9DD>9:B<	AS:i:-1	ZS:i:-1	XN:i:0	XM:i:0	XO:i:0	XG:i:0	NM:i:0	MD:Z:49	YT:Z:UU	NH:i:4
SRR9048103.33222634	256	1	14399	1	50M	*	0	0	GTTGGTTTCNGCTCAGTTCTTTATTGATTGGTGTGCCGTTTTCTCTGGAA	?@:BDADDH#2ADGIIIIIIIIGIIIIIIEG?1CFGGIHEGGIIBB<*?F	AS:i:-2	ZS:i:-2	XN:i:0	XM:i:1	XO:i:0	XG:i:0	NM:i:1	MD:Z:9T40	YT:Z:UU	NH:i:5
SRR9048103.30258511	256	1	14403	1	50M	*	0	0	GTTTCTGCTA

# Quantify gene counts

Using featureCounts we can count how many reads there have been per gene.

For this we call featureCounts with default parameters and ask featureCounts to assess counts per -t exon and summarize by -g gene_id . As reference we hand over gencode v39 human reference. We specify an output file and all bam-files.

First have a quick glance at the annotation file:

In [10]:
! cat ./references/ensembl.105.annotation.gtf | head

##description: evidence-based annotation of the human genome (GRCh38), version 39 (Ensembl 105)
##provider: GENCODE
##contact: gencode-help@ebi.ac.uk
##format: gtf
##date: 2021-09-02
chr1	HAVANA	gene	11869	14409	.	+	.	gene_id "ENSG00000223972.5"; gene_type "transcribed_unprocessed_pseudogene"; gene_name "DDX11L1"; level 2; hgnc_id "HGNC:37102"; havana_gene "OTTHUMG00000000961.2";
chr1	HAVANA	transcript	11869	14409	.	+	.	gene_id "ENSG00000223972.5"; transcript_id "ENST00000456328.2"; gene_type "transcribed_unprocessed_pseudogene"; gene_name "DDX11L1"; transcript_type "processed_transcript"; transcript_name "DDX11L1-202"; level 2; transcript_support_level "1"; hgnc_id "HGNC:37102"; tag "basic"; havana_gene "OTTHUMG00000000961.2"; havana_transcript "OTTHUMT00000362751.1";
chr1	HAVANA	exon	11869	12227	.	+	.	gene_id "ENSG00000223972.5"; transcript_id "ENST00000456328.2"; gene_type "transcribed_unprocessed_pseudogene"; gene_name "DDX11L1"; transcript_type "processed_transcript"; transcript_n

This file tells where on the genome a specific gene (which consists of several transcript isoforms which again consist of several exons) is located. Then it is checked whether a read falls into a known exon. If so, this exon count is incremented by one. Finally, all exons of a gene are aggregated.

In [24]:
! cd ./alignments/bulk/ &&  ../../tools/subread-2.0.3-source/bin/featureCounts -T 4 -t exon -g gene_id -a ../../references/ensembl.105.annotation.gtf -o counts.featurecounts.hisat2.tsv SUM159_24h_R1.bam SUM159_24h_R2.bam SUM159R_24h_R1.bam SUM159R_24h_R2.bam


        ==========     _____ _    _ ____  _____  ______          _____  
        =====         / ____| |  | |  _ \|  __ \|  ____|   /\   |  __ \ 
          =====      | (___ | |  | | |_) | |__) | |__     /  \  | |  | |
            ====      \___ \| |  | |  _ <|  _  /|  __|   / /\ \ | |  | |
              ====    ____) | |__| | |_) | | \ \| |____ / ____ \| |__| |
        ==========   |_____/ \____/|____/|_|  \_\______/_/    \_\_____/
	  v2.0.3

//========================== featureCounts setting ===========================\\
||                                                                            ||
||             Input files : 4 BAM files                                      ||
||                                                                            ||
||                           SUM159_24h_R1.bam                                ||
||                           SUM159_24h_R2.bam                                ||
||                           SUM159R_24h_R1.bam                  

This summary tells us how well the mapping could be assigned to read-counts. Some reads could not be aligned due to multimapping - possibly because the read length was chosen to be too short.

In [25]:
! cat ./alignments/bulk/counts.featurecounts.hisat2.tsv.summary

Status	SUM159_24h_R1.bam	SUM159_24h_R2.bam	SUM159R_24h_R1.bam	SUM159R_24h_R2.bam
Assigned	12730408	29461647	31964377	20473798
Unassigned_Unmapped	1101887	1926278	1944297	1520084
Unassigned_Read_Type	0	0	0	0
Unassigned_Singleton	0	0	0	0
Unassigned_MappingQuality	0	0	0	0
Unassigned_Chimera	0	0	0	0
Unassigned_FragmentLength	0	0	0	0
Unassigned_Duplicate	0	0	0	0
Unassigned_MultiMapping	6488761	15144678	15243105	10011071
Unassigned_Secondary	0	0	0	0
Unassigned_NonSplit	0	0	0	0
Unassigned_NoFeatures	765478	1736538	1548873	1132298
Unassigned_Overlapping_Length	0	0	0	0
Unassigned_Ambiguity	1102693	2602009	2799429	1726337
